# EE399 HW4
## Ziwen


https://github.com/ZiwenLi0325/EE399.git

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy import integrate
from mpl_toolkits.mplot3d import Axes3D
import torch
from torch import nn, optim
from torch.optim import Adam
from sklearn.metrics import mean_squared_error

In [31]:
dt = 0.01
T = 8
t = np.arange(0,T+dt,dt)
beta = 8/3
sigma = 10
rho = 28

# Define activation functions
def logsig(x):
    return 1 / (1 + torch.exp(-x))

def radbas(x):
    return torch.exp(-torch.pow(x, 2))

def purelin(x):
    return x

# Define the Lorenz system
def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
    x, y, z = x_y_z
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 3)

    def forward(self, x):
        x = purelin(self.fc1(x))
        x = purelin(self.fc2(x))
        x = purelin(self.fc3(x))
        return x

class FeedForwardNN(nn.Module):
    def __init__(self):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(3, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.bn = nn.BatchNorm1d(hidden_size)  # add batch normalization
        self.dropout = nn.Dropout(0.5)  # dropout layer

    def forward(self, x):
        x, _ = self.rnn(x.unsqueeze(1))  # add an extra dimension for timesteps
        x = self.dropout(x)  # add dropout
        x = self.fc(self.bn(x.squeeze(1)))  # apply batch normalization before fc
        return x
    
    
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, dropout_rate=0.5):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x, _ = self.lstm(x.unsqueeze(1))  # add an extra dimension for timesteps
        x = self.dropout(x)
        x = self.fc(x.squeeze(1))  # remove the timesteps dimension
        return x



class ESN(nn.Module):
    def __init__(self, input_size, reservoir_size, output_size, alpha=0.5):
        super(ESN, self).__init__()
        self.input_weights = nn.Parameter(torch.randn(reservoir_size, input_size) / np.sqrt(input_size), requires_grad=False)
        self.reservoir_weights = nn.Parameter(torch.randn(reservoir_size, reservoir_size) / np.sqrt(reservoir_size), requires_grad=False)
        self.output_weights = nn.Linear(reservoir_size, output_size)

        spectral_radius = np.max(np.abs(np.linalg.eigvals(self.reservoir_weights.detach().numpy())))
        self.reservoir_weights.data = self.reservoir_weights.data / spectral_radius * alpha
        self.reservoir_size = reservoir_size

    def forward(self, input):
        reservoir_state = torch.zeros(input.size(0), input.size(1), self.reservoir_size, dtype=torch.float32, device=input.device)
        reservoir_state[:, 0, :] = torch.tanh(torch.mm(input[:,0,:], self.input_weights.t()))  # initialize reservoir state at t=0
        for t in range(1, input.size(1)):  # start loop from t=1
            reservoir_state[:, t, :] = torch.tanh(torch.mm(input[:,t,:], self.input_weights.t()) + torch.mm(reservoir_state[:, t-1, :], self.reservoir_weights.t()))
        output = torch.sigmoid(self.output_weights(reservoir_state))  # changed activation function
        return output


# train for rho = 10, 28, 40

## nn

In [4]:
# Generate training data for rho=10, 28, 40
rhos = [10, 28, 40]
training_input = []
training_output = []
for rho in rhos:
    np.random.seed(123)
    x0 = -15 + 30 * np.random.random((100, 3))

    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t, args=(sigma, beta, rho)) for x0_j in x0])
    
    for j in range(100):
        training_input.append(x_t[j,:-1,:])
        training_output.append(x_t[j,1:,:])

training_input = np.vstack(training_input)
training_output = np.vstack(training_output)

# Convert numpy arrays to PyTorch tensors
training_input_torch = torch.tensor(training_input, dtype=torch.float32)
training_output_torch = torch.tensor(training_output, dtype=torch.float32)

# Initialize the model and optimizer
model_nn_10 = Net()
optimizer = Adam(model_nn_10.parameters())

# Define the loss function
criterion = nn.MSELoss()

# Train the network
for epoch in range(50):  # 50 epochs
    optimizer.zero_grad()   # zero the gradient buffers
    output = model_nn_10(training_input_torch)
    loss = criterion(output, training_output_torch)
    loss.backward()
    optimizer.step()    # Does the update
    if epoch % 2 == 0:
        print('For NN : Epoch: {}, Loss: {:.5f}'.format(epoch, loss.item()))


For NN : Epoch: 0, Loss: 265.38974
For NN : Epoch: 2, Loss: 221.21327
For NN : Epoch: 4, Loss: 181.20503
For NN : Epoch: 6, Loss: 145.34227
For NN : Epoch: 8, Loss: 113.58848
For NN : Epoch: 10, Loss: 85.93810
For NN : Epoch: 12, Loss: 62.42421
For NN : Epoch: 14, Loss: 43.10167
For NN : Epoch: 16, Loss: 28.00419
For NN : Epoch: 18, Loss: 17.07340
For NN : Epoch: 20, Loss: 10.06859
For NN : Epoch: 22, Loss: 6.48238
For NN : Epoch: 24, Loss: 5.51412
For NN : Epoch: 26, Loss: 6.15774
For NN : Epoch: 28, Loss: 7.40201
For NN : Epoch: 30, Loss: 8.45557
For NN : Epoch: 32, Loss: 8.88256
For NN : Epoch: 34, Loss: 8.59652
For NN : Epoch: 36, Loss: 7.75220
For NN : Epoch: 38, Loss: 6.61079
For NN : Epoch: 40, Loss: 5.43061
For NN : Epoch: 42, Loss: 4.40072
For NN : Epoch: 44, Loss: 3.61710
For NN : Epoch: 46, Loss: 3.09213
For NN : Epoch: 48, Loss: 2.78266


## FeedForwardNN: rho = 10

In [5]:

# Initialize the model and optimizer
model_FFNN_10 = FeedForwardNN()
optimizer = Adam(model_FFNN_10.parameters())

# Define the loss function
criterion = nn.MSELoss()

# Train the network
for epoch in range(50):  # 100 epochs
    optimizer.zero_grad()   # zero the gradient buffers
    output = model_FFNN_10(training_input_torch)
    loss = criterion(output, training_output_torch)
    loss.backward()
    optimizer.step()    # Does the update
    if epoch % 2 == 0:
        print('For FeedForwardNN: Epoch: {}, Loss: {:.5f}'.format(epoch, loss.item()))

For FeedForwardNN: Epoch: 0, Loss: 328.90268
For FeedForwardNN: Epoch: 2, Loss: 304.58813
For FeedForwardNN: Epoch: 4, Loss: 282.76456
For FeedForwardNN: Epoch: 6, Loss: 263.17981
For FeedForwardNN: Epoch: 8, Loss: 245.04854
For FeedForwardNN: Epoch: 10, Loss: 227.95808
For FeedForwardNN: Epoch: 12, Loss: 211.64238
For FeedForwardNN: Epoch: 14, Loss: 195.94267
For FeedForwardNN: Epoch: 16, Loss: 180.79012
For FeedForwardNN: Epoch: 18, Loss: 166.12222
For FeedForwardNN: Epoch: 20, Loss: 151.79556
For FeedForwardNN: Epoch: 22, Loss: 137.70491
For FeedForwardNN: Epoch: 24, Loss: 123.84919
For FeedForwardNN: Epoch: 26, Loss: 110.29974
For FeedForwardNN: Epoch: 28, Loss: 97.00536
For FeedForwardNN: Epoch: 30, Loss: 84.04802
For FeedForwardNN: Epoch: 32, Loss: 71.55025
For FeedForwardNN: Epoch: 34, Loss: 59.68688
For FeedForwardNN: Epoch: 36, Loss: 48.63945
For FeedForwardNN: Epoch: 38, Loss: 38.58652
For FeedForwardNN: Epoch: 40, Loss: 29.74803
For FeedForwardNN: Epoch: 42, Loss: 22.31814
F

## SimpleRNN: rho = 10

In [13]:
# Generate training data for rho=10, 28, 40
training_input = []
training_output = []

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
model_rnn = SimpleRNN(3, 50, 3)  # input size = 3, hidden size = 50, output size = 3
optimizer = Adam(model_rnn.parameters(), lr=0.01)  # adjust the learning rate if necessary

model_rnn = SimpleRNN(3, 50, 3)  # input size = 3, hidden size = 50, output size = 3
optimizer = torch.optim.RMSprop(model_rnn.parameters(), lr=0.01)  # try RMSprop optimizer

# Define the loss function
criterion = nn.MSELoss()

# Train the network
best_loss = np.inf
epochs_no_improve = 0

for epoch in range(50):  # 30 epochs
    optimizer.zero_grad()   # zero the gradient buffers
    output = model_rnn(training_input_torch)
    loss = criterion(output, training_output_torch)

    # Add L2 regularization
    l2_reg = None
    for W in model_rnn.parameters():
        if l2_reg is None:
            l2_reg = W.norm(2)
        else:
            l2_reg = l2_reg + W.norm(2)
    loss += 0.01 * l2_reg

    loss.backward()

    # Gradient clipping
    torch.nn.utils.clip_grad_norm_(model_rnn.parameters(), max_norm=1)

    optimizer.step()    # Does the update

    # Update learning rate
    scheduler.step()

    # Early stopping
    if loss.item() < best_loss:
        epochs_no_improve = 0
        best_loss = loss.item()
    else:
        epochs_no_improve += 1
        # Check early stopping condition
        if epochs_no_improve == 5:
            print('Early stopping!')
            break

    if epoch % 2 == 0:
        print('For SimpleRNN: Epoch: {}, Loss: {:.5f}'.format(epoch, loss.item()))

For SimpleRNN: Epoch: 0, Loss: 294.86755
For SimpleRNN: Epoch: 2, Loss: 239.81398
For SimpleRNN: Epoch: 4, Loss: 187.93257
For SimpleRNN: Epoch: 6, Loss: 150.30766
For SimpleRNN: Epoch: 8, Loss: 120.57673
For SimpleRNN: Epoch: 10, Loss: 100.68346
For SimpleRNN: Epoch: 12, Loss: 63.49220
For SimpleRNN: Epoch: 14, Loss: 53.25209
For SimpleRNN: Epoch: 16, Loss: 39.12949
For SimpleRNN: Epoch: 18, Loss: 34.01862
For SimpleRNN: Epoch: 20, Loss: 33.48181
For SimpleRNN: Epoch: 22, Loss: 22.82074
For SimpleRNN: Epoch: 24, Loss: 22.65281
For SimpleRNN: Epoch: 26, Loss: 22.64660
For SimpleRNN: Epoch: 28, Loss: 23.14999
For SimpleRNN: Epoch: 30, Loss: 24.63207
For SimpleRNN: Epoch: 32, Loss: 22.14604
For SimpleRNN: Epoch: 34, Loss: 23.61559
For SimpleRNN: Epoch: 36, Loss: 23.84859
For SimpleRNN: Epoch: 38, Loss: 25.66619
For SimpleRNN: Epoch: 40, Loss: 23.56945
For SimpleRNN: Epoch: 42, Loss: 21.98042
For SimpleRNN: Epoch: 44, Loss: 21.90574
For SimpleRNN: Epoch: 46, Loss: 21.89048
For SimpleRNN: 

## LSTM:

In [33]:
# Initialize the model and optimizer
model_LSTM = LSTM(3, 100, 3)  # input size = 3, hidden size = 100, output size = 3
optimizer = torch.optim.RMSprop(model_LSTM.parameters(), lr=0.01)

# Define the loss function
criterion = nn.MSELoss()

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Train the network
for epoch in range(30):  # 30 epochs
    optimizer.zero_grad()   # zero the gradient buffers
    output = model_LSTM(training_input_torch)
    loss = criterion(output, training_output_torch)
    loss.backward()
    optimizer.step()    # Does the update

    # Step the scheduler
    scheduler.step()

    print('For LSTM : Epoch: {}, Loss: {:.5f}'.format(epoch, loss.item()))

For LSTM : Epoch: 0, Loss: 293.70132
For LSTM : Epoch: 1, Loss: 241.11096
For LSTM : Epoch: 2, Loss: 180.30966
For LSTM : Epoch: 3, Loss: 143.97894
For LSTM : Epoch: 4, Loss: 115.30595
For LSTM : Epoch: 5, Loss: 103.44151
For LSTM : Epoch: 6, Loss: 91.05454
For LSTM : Epoch: 7, Loss: 78.71951
For LSTM : Epoch: 8, Loss: 78.42130
For LSTM : Epoch: 9, Loss: 74.51981
For LSTM : Epoch: 10, Loss: 87.20402
For LSTM : Epoch: 11, Loss: 76.40270
For LSTM : Epoch: 12, Loss: 71.33307
For LSTM : Epoch: 13, Loss: 68.19150
For LSTM : Epoch: 14, Loss: 66.08487
For LSTM : Epoch: 15, Loss: 64.27159
For LSTM : Epoch: 16, Loss: 63.15765
For LSTM : Epoch: 17, Loss: 62.03322
For LSTM : Epoch: 18, Loss: 61.33126
For LSTM : Epoch: 19, Loss: 60.37132
For LSTM : Epoch: 20, Loss: 59.65639
For LSTM : Epoch: 21, Loss: 59.62489
For LSTM : Epoch: 22, Loss: 59.54759
For LSTM : Epoch: 23, Loss: 59.51046
For LSTM : Epoch: 24, Loss: 59.44521
For LSTM : Epoch: 25, Loss: 59.39427
For LSTM : Epoch: 26, Loss: 59.41616
For L

## ESN:

In [36]:
# Add an extra dimension for time step to the input tensor
training_input_torch_time = training_input_torch.unsqueeze(1)

# Define the models
model_esn_10 = ESN(3, 100, 3)  # increased reservoir size

# Initialize the optimaizer for ESN
optimizer_esn = torch.optim.Adam(model_esn_10.output_weights.parameters(), lr=0.01)

# Initialize the optimizer for ESN
optimizer_esn = torch.optim.RMSprop(model_esn_10.output_weights.parameters(), lr=0.01)  # changed optimizer

# Define the loss function
criterion = nn.MSELoss()

# Train the ESN network
best_loss_esn = np.inf
epochs_no_improve_esn = 0

for epoch in range(30):  # match the number of epochs with NN
    optimizer_esn.zero_grad()   # zero the gradient buffers
    output_esn = model_esn_10(training_input_torch_time)
    # Remove the time step dimension from the output for loss calculation
    output_esn = output_esn.squeeze(1)
    loss_esn = criterion(output_esn, training_output_torch)

    # Add L2 regularization
    l2_reg = None
    for W in model_esn_10.output_weights.parameters():
        if l2_reg is None:
            l2_reg = W.norm(2)
        else:
            l2_reg = l2_reg + W.norm(2)
    loss_esn += 0.01 * l2_reg  # L2 regularization

    loss_esn.backward()
    optimizer_esn.step()  # Does the update

    # Early stopping
    if loss_esn.item() < best_loss_esn:
        epochs_no_improve_esn = 0
        best_loss_esn = loss_esn.item()
    else:
        epochs_no_improve_esn += 1
        # Check early stopping condition
        if epochs_no_improve_esn == 5:
            print('Early stopping!')
            break

    print('For ESN : Epoch: {}, Loss: {:.5f}'.format(epoch, loss_esn.item()))


For ESN : Epoch: 0, Loss: 293.55878
For ESN : Epoch: 1, Loss: 180.07312
For ESN : Epoch: 2, Loss: 128.10544
For ESN : Epoch: 3, Loss: 99.24440
For ESN : Epoch: 4, Loss: 83.95939
For ESN : Epoch: 5, Loss: 74.97944
For ESN : Epoch: 6, Loss: 69.38313
For ESN : Epoch: 7, Loss: 65.77721
For ESN : Epoch: 8, Loss: 63.39189
For ESN : Epoch: 9, Loss: 61.77143
For ESN : Epoch: 10, Loss: 60.63675
For ESN : Epoch: 11, Loss: 59.81379
For ESN : Epoch: 12, Loss: 59.19267
For ESN : Epoch: 13, Loss: 58.70356
For ESN : Epoch: 14, Loss: 58.30184
For ESN : Epoch: 15, Loss: 57.95888
For ESN : Epoch: 16, Loss: 57.65624
For ESN : Epoch: 17, Loss: 57.38194
For ESN : Epoch: 18, Loss: 57.12825
For ESN : Epoch: 19, Loss: 56.89003
For ESN : Epoch: 20, Loss: 56.66390
For ESN : Epoch: 21, Loss: 56.44754
For ESN : Epoch: 22, Loss: 56.23936
For ESN : Epoch: 23, Loss: 56.03824
For ESN : Epoch: 24, Loss: 55.84336
For ESN : Epoch: 25, Loss: 55.65408
For ESN : Epoch: 26, Loss: 55.46994
For ESN : Epoch: 27, Loss: 55.29048

In [37]:
rhos = [17, 35]

def generate_lorenz_data(rho, initial_state=[1, 1, 1], dt=0.01, N=10000, sigma=10, beta=8/3):
    t = np.arange(0, N*dt, dt)
    traj = integrate.odeint(lorenz_deriv, initial_state, t, args=(sigma, beta, rho))
    return traj[:-1, :], traj[1:, :]

for rho in rhos:
    print(f"\nForecasting dynamics for rho = {rho}:")

    # Generate Lorenz system data for rho
    nn_input, nn_output = generate_lorenz_data(rho)

    # Train-test split
    train_frac = 0.8
    split_idx = int(train_frac * len(nn_input))
    train_input, train_output = nn_input[:split_idx], nn_output[:split_idx]
    test_input, test_output = nn_input[split_idx:], nn_output[split_idx:]

    # Convert to torch tensors
    train_input_torch = torch.from_numpy(train_input.astype(np.float32))
    train_output_torch = torch.from_numpy(train_output.astype(np.float32))
    test_input_torch = torch.from_numpy(test_input.astype(np.float32))
    test_output_torch = torch.from_numpy(test_output.astype(np.float32))

    # Define the models
    model1 = model_nn_10
    model2 = model_FFNN_10
    model3 = model_LSTM
    model4 = model_rnn
    model5 = model_esn_10
  

    # Define the loss function
    criterion = nn.MSELoss()

    # Train each model and make predictions
    for model in [model1, model2, model3, model4]:
        # Define the optimizer
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

        # Train the model
        for epoch in range(100):
            optimizer.zero_grad()
            output = model(train_input_torch)
            loss = criterion(output, train_output_torch)
            loss.backward()
            optimizer.step()

        # Make predictions on the test data
        model.eval()
        with torch.no_grad():
            predictions = model(test_input_torch)

        # Compute the mean squared error of the predictions
        mse = mean_squared_error(test_output_torch.detach().numpy(), predictions.detach().numpy())
        print(f"Mean Squared Error for model {model.__class__.__name__}: {mse}")
    


Forecasting dynamics for rho = 17:
Mean Squared Error for model Net: 0.00046261821989901364
Mean Squared Error for model FeedForwardNN: 7.882964382588398e-06
Mean Squared Error for model LSTM: 0.0024459792766720057
Mean Squared Error for model SimpleRNN: 0.018040049821138382

Forecasting dynamics for rho = 35:
Mean Squared Error for model Net: 0.3871428668498993
Mean Squared Error for model FeedForwardNN: 0.09199709445238113
Mean Squared Error for model LSTM: 0.4567260444164276
Mean Squared Error for model SimpleRNN: 0.17212016880512238


In [38]:
for rho in rhos:
    print(f"\nForecasting dynamics for rho = {rho}:")

    # Generate Lorenz system data for rho
    nn_input, nn_output = generate_lorenz_data(rho)

    # Train-test split
    train_frac = 0.8
    split_idx = int(train_frac * len(nn_input))
    train_input, train_output = nn_input[:split_idx], nn_output[:split_idx]
    test_input, test_output = nn_input[split_idx:], nn_output[split_idx:]

    # Convert to torch tensors and add time dimension
    train_input_torch = torch.from_numpy(train_input.astype(np.float32)).unsqueeze(1)
    train_output_torch = torch.from_numpy(train_output.astype(np.float32))
    test_input_torch = torch.from_numpy(test_input.astype(np.float32)).unsqueeze(1)
    test_output_torch = torch.from_numpy(test_output.astype(np.float32))

    # Define the models
    model5 = model_esn_10

    # Define the loss function
    criterion = nn.MSELoss()

    # Train each model and make predictions
    for model in [model5]:
        # Define the optimizer
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

        # Train the model
        for epoch in range(100):
            optimizer.zero_grad()
            output = model(train_input_torch)
            output = output.squeeze(1)  # remove the time dimension for ESN output
            loss = criterion(output, train_output_torch)
            loss.backward()
            optimizer.step()

        # Make predictions on the test data
        model.eval()
        with torch.no_grad():
            predictions = model(test_input_torch)
            predictions = predictions.squeeze(1)  # remove the time dimension for ESN predictions

        # Compute the mean squared error of the predictions
        mse = mean_squared_error(test_output_torch.detach().numpy(), predictions.detach().numpy())
        print(f"Mean Squared Error for model {model.__class__.__name__}: {mse}")



Forecasting dynamics for rho = 17:
Mean Squared Error for model ESN: 0.0032536678481847048

Forecasting dynamics for rho = 35:
Mean Squared Error for model ESN: 23.312599182128906
